<br></br>
# **나이브베이즈 한글 적용**
nltk를 활용한 네이버 영화리뷰 평가모델 [nltk book](https://www.nltk.org/book/ch06.html) | [nltk How to](http://www.nltk.org/howto/classify.html)
1. 0 : 부정적인 리뷰
1. 1 : 긍정적인 리뷰

<br></br>
## **1 NAVER 영화리뷰 데이터 전처리**
1. https://github.com/e9t/nsmc
1. https://www.nltk.org/book/ch06.html

In [ ]:
# ! cat ./data/ratings_test.txt | head -n 10

In [ ]:
def read_data(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        data = [line.split('\t') for line in f.read().splitlines()]
        data = data[1:]   # header 제외
        
    from random import randint
    random_data = [data[randint(1, len(data))]  for no in range(int(len(data)/20)) ]
    return random_data

In [ ]:
train_data = read_data('./data/ratings_train.txt')
test_data  = read_data('./data/ratings_test.txt')

print('Train_data ({})\nsample : {}\nTest_data  ({})\nsample : {}'.format(
    len(train_data), train_data[:3],
    len(test_data),  test_data[:3]))

In [ ]:
%%time
from konlpy.tag import Twitter
twitter = Twitter()

def tokenize(doc):
    result = ['/'.join(t) for t in twitter.pos(doc, norm=True, stem=True)]
    return result 

train_docs = [(tokenize(row[1]), row[2])    for row in train_data]
test_docs  = [(tokenize(row[1]), row[2])    for row in test_data]

from pprint import pprint
pprint(train_docs[:2])

In [ ]:
tokens = [t   for d in train_docs 
              for t in d[0]]
print("Token Total :{}\nSample : {}".format(
    len(tokens), tokens[:5]))

<br></br>
## **2 nltk 를 활용하여 연산모델 만들기**
https://github.com/e9t/nsmc

In [ ]:
import nltk
text = nltk.Text(tokens, name='네이버리뷰')

print("number of Token : {} \nunique Token    : {}\n".format(
    len(text.tokens), len(set(text.tokens))))
pprint(text.vocab().most_common(20))    

In [ ]:
%matplotlib inline
from matplotlib import rc
rc('font', family=['NanumGothic','Malgun Gothic'])

import matplotlib.pyplot as plt
plt.figure(figsize=(16,5))
text.plot(50) 

<br></br>
## **3 모델의 정확도/ 일반화를 높이는 추가작업**
우도 상위 4000개 데이터를 추출

In [ ]:
# 검색가능 단어목록 추출하기
selected_words = [f[0] for f in text.vocab().most_common(4000)]
selected_words[:5]

In [ ]:
# Mission 1
# selected 객체를 './data/selected.words' 로 저장하기
import pickle
# pickle.dump(selected_words, open('./data/selected.words', 'wb'))

In [ ]:
%%time
def term_exists(doc):
    return {'exists({})'.format(word): (word in set(doc)) for word in selected_words}

train_xy   = [(term_exists(d), c) for d, c in train_docs]
test_xy    = [(term_exists(d), c) for d, c in test_docs]

In [ ]:
# 문장별 Text을 분석가능한 Token으로 변환
line_num = 130
print("100th 문장 변환된 Token 갯수 : {}\n\nSample : {}".format(
    len(train_xy[line_num][0]),
    train_xy[line_num][0]))

In [ ]:
%%time
classifiers = nltk.NaiveBayesClassifier.train(train_xy)

In [ ]:
# Mission 2
# classifiers 객체를 './data/classifiers.model' 로 저장하기
import pickle
# pickle.dump(classifiers, open('./data/classifiers.model', 'wb'))

<br></br>
## **4 생성한 모델을 평가**
Accuracy
1. 0 : 부정리뷰
1. 1 : 긍정리뷰

In [ ]:
%%time
import pickle
# 학습한 모델객체 저장하기
# Google 에 찾아보기 : Python classifier Save

In [ ]:
classifiers.labels()

In [ ]:
classifiers.show_most_informative_features(20)

In [ ]:
%%time
# Test 데이터로 Model의 정확도 측정
'네이버 영화리뷰 모델의 Accuracy : {}'.format(
    nltk.classify.accuracy(classifiers, test_xy))

<br></br>
## **5 모델의 활용**
1. 0 : 부정리뷰
1. 1 : 긍정리뷰

In [ ]:
review = """졸잼 굿 최고입니다 최고"""

In [ ]:
# 리뷰 데이터를 Tagged Token 생성하기
review = tokenize(review)    
review

In [ ]:
# Tagged Token 중 selected_words 포함여부 판단
review = term_exists(review) 
for k, v in review.items():
    if v == True:
        print("{} = {}".format(k, v))

In [ ]:
result = classifiers.classify(review)  # 분류모델 평가
if result == '1': 
    print('긍정리뷰')
else:             
    print('부정리뷰')